In [1]:
import cv2
import numpy as np
import pandas as pd
import os
import sys
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
import keras_tuner as kt
from sklearn.metrics import classification_report
import shutil
from os import path
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image_dataset_from_directory
from keras.preprocessing.image import ImageDataGenerator

# path variables to access files
root = os.getcwd()

### Load dataframe

In [2]:
main_data = pd.read_csv("data_labels_mainData.csv")
extra_data = pd.read_csv("data_labels_extraData.csv")

### Get data from splitted folder

In [3]:
# method get data for multiclass task (task 2)
def celltype_classify_data(class_list):    
    images = list()
    labels = list()
    
    for i, label in enumerate(class_list):
        # get image directory
        img_dir = os.path.join(root, "multiclass_task", f"{label}")
        
        for img in os.listdir(img_dir):
            img = cv2.imread(os.path.join(img_dir, img))
            # resize to 0-1 for faster computation
            resized = img / 255
            images.append(resized)
            labels.append(i)
        
    return (images, labels)

### Classification report method

In [4]:
def multiclass_classification_report(y_test, prediction):
    """
        Method to generate sklearn classification report with CNN multiclass output
    """
    
    encoded_pred = list()
    # convert each CNN output (sparse categorial) to class
    for pred in prediction:
        encoded_pred.append(np.argmax(pred))

    encoded_pred = np.array(encoded_pred)
    print(classification_report(y_test, encoded_pred))

### Structure of sub-models

In [7]:
def get_multiclass_model():
    """
        Structure of model classifying 4 classes
    """
    
    model = tf.keras.Sequential()
    # First convo-pooling
    # Convolutional layers (filter the image with a kernel)
    model.add(tf.keras.layers.Conv2D(64, (3, 3), strides=1,activation="relu", input_shape=[27, 27, 3]))
    # Max-pooling layers (reduce the size of the image by choosing max pixel at certain area)
    model.add(tf.keras.layers.MaxPool2D(pool_size=(2, 2), strides=1))
    
    # Second convo-pooling
    # Convolutional layers (filter the image with a kernel)
    model.add(tf.keras.layers.Conv2D(64, (3, 3),strides=1, activation="relu"))
    # Max-pooling layers (reduce the size of the image by choosing max pixel at certain area)
    model.add(tf.keras.layers.MaxPool2D(pool_size=(2, 2),strides=1))
    
    # Flatten input
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(2, activation="relu"))
    model.add(tf.keras.layers.Dense(128, activation="relu"))
    model.add(tf.keras.layers.Dense(32, activation="relu"))
    # Output layer
    model.add(tf.keras.layers.Dense(4, activation="softmax"))
    # Compile model 
    model.compile(optimizer="adam",
                  loss="sparse_categorical_crossentropy",
                  metrics=["accuracy"])
    
    return model

def get_subclass_model():
    """
        Structure of model classifying 3 classes
    """
    
    model = tf.keras.Sequential()
    # First convo-pooling
    # Convolutional layers (filter the image with a kernel)
    model.add(tf.keras.layers.Conv2D(64, (3, 3), activation="relu", input_shape=[27, 27, 3]))
    # Max-pooling layers (reduce the size of the image by choosing max pixel at certain area)
    model.add(tf.keras.layers.MaxPool2D(pool_size=(2, 2)))
    # Flatten input
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(256, activation="relu"))
    model.add(tf.keras.layers.Dense(128, activation="relu"))
    model.add(tf.keras.layers.Dense(32, activation="relu"))
    # Output layer
    model.add(tf.keras.layers.Dense(3, activation="softmax"))
    # Compile model 
    model.compile(optimizer="adam",
                  loss="sparse_categorical_crossentropy",
                  metrics=["accuracy"])
    
    return model

### Methods to train and save submodels

In [55]:
submodel_path = os.path.join(root, "multiclass_submodels")   
def three_class_submodel(file_path, aug_train = None, aug_val = None):
    # path to model trained with 3 classes
    subclass_path = path.join(file_path, "subclass.h5")
    subclass_model = None
    if not os.path.isfile(subclass_path) or keras.models.load_model(subclass_path) == None:
        print("---Sub model training---")
        # train subclass that fit with 3 types of images
        subclass_model = get_subclass_model()
        
        if aug_train is not None and aug_val is not None:
            print("Train with augmented data")
            subclass_model.fit(aug_train, epochs=40, validation_data=aug_val)
        else:
            print("Train with non-augmented data")
            subclass_model.fit(subx_train, suby_train, epochs=40, validation_data=(subx_test, suby_test))
            
        subclass_model.save(subclass_path)
        return subclass_model
    else:
        subclass_model = keras.models.load_model(subclass_path)
        print("Model trained with 3 classes loaded")
        return subclass_model
    
def all_class_submodel(file_path, aug_train = None, aug_val = None):
    allclass_path = path.join(file_path, "allclass.h5")
    allclass_model = None
    if not os.path.isfile(allclass_path) or keras.models.load_model(allclass_path) == None:
        print("---Sub model training---")
        # train subclass that fit with 4 types of images
        allclass_model = get_multiclass_model()
        
        if aug_train is not None and aug_val is not None:
            print("Train with augmented data")
            allclass_model.fit(aug_train, epochs=40, validation_data=aug_val)
        else:
            print("Train with non-augmented data")
            allclass_model.fit(x_train, y_train, epochs=15, validation_data=(x_val, y_val))
            
        allclass_model.save(allclass_path)
        return allclass_model
    else:
        allclass_model = keras.models.load_model(allclass_path)
        print("Model trained with 4 classes loaded")
        return allclass_model

### Meta-learner

In [30]:
def get_transfer_model():
    model = tf.keras.Sequential()
    # Flatten input
    model.add(tf.keras.layers.Flatten())
    # Hidden layers
    model.add(tf.keras.layers.Dense(256, activation="relu"))
    model.add(tf.keras.layers.Dense(128, activation="softmax"))
    model.add(tf.keras.layers.Dense(32, activation="relu"))
    model.add(tf.keras.layers.Dense(3, activation="sigmoid"))
    # Output layer
    model.add(tf.keras.layers.Dense(4, activation="softmax"))
    # Compile model
    model.compile(optimizer="adam",
                  loss="sparse_categorical_crossentropy",
                  metrics=["accuracy"])
    return model

### Get augmentated data for training

In [56]:
# load images from folders
all_class = ["epithelial", "fibroblast", "inflammatory", "others"]
cell_img, cell_label = celltype_classify_data(all_class)
cell_img, cell_label = np.array(cell_img), np.array(cell_label)

sub_multiclass = ["epithelial", "fibroblast", "inflammatory"]
sub_img, sub_label = celltype_classify_data(sub_multiclass)

# train and validate data for 3 classes
subx_train, subx_test, suby_train, suby_test = train_test_split(
    np.array(sub_img), np.array(sub_label), test_size=0.2
)

# train and validate data for 4 classes
x_train_val, x_test, y_train_val, y_test = train_test_split(
    cell_img, cell_label, test_size=0.2
)
x_train, x_val, y_train, y_val = train_test_split(
    x_train_val, y_train_val, test_size=0.25
)

# get augmentation for extra training data
extra_datagen = ImageDataGenerator(
    width_shift_range=0.3,
    height_shift_range=0.3,
    rotation_range=60,
    horizontal_flip=True,
    vertical_flip=True,

)

# fit augmentation with all images from all 4 classes
extra_datagen.fit(cell_img)

# get augmentations for unlabeled data
shift_aug = ImageDataGenerator(
    width_shift_range=0.3,
    height_shift_range=0.3,
)
rotation_aug = ImageDataGenerator(
    rotation_range=60
)
flip_aug = ImageDataGenerator(
    horizontal_flip=True,
    vertical_flip=True,
)
aug_list = [shift_aug, rotation_aug, flip_aug]

# git aug_list with images
for aug in aug_list:
    aug.fit(cell_img)

### Helper function 

In [57]:
def get_extra_stacked(x, submodels, datagen):
    """Generate stacked output from submodels with augmented x for meta learner"""
    stacked = None
    # augmentate x before predicting with submodels
    pred_gen = datagen.flow(x, shuffle=False)
    for submodel in submodels:
        if stacked is None:
            stacked = submodel.predict_generator(pred_gen)
        else:
            # augmentate x before predicting with submodels
            new_pred = submodel.predict_generator(pred_gen)
            stacked = np.concatenate((stacked, new_pred), axis=1)
    return stacked

def mix_match_pred(x, aug_list, sub_models, meta_learner):
#     get prediction of mix match model
    pred_arr = None
#     Find the sum of all predictions from each augmentation test set
    for aug in aug_list:
        # get stacked output from submodels to let meta_learner predict 
        stacked_x = get_extra_stacked(x, sub_models, aug)
        # predict with metta_learner
        pred = meta_learner.predict(stacked_x)
        
        if pred_arr is None:
            pred_arr = pred
        else:
            pred_arr = np.add(pred_arr, pred)
#     Find the average of prediction      
    pred_arr = np.true_divide(pred_arr, len(aug_list))
    return pred_arr

In [58]:
extramodel_path = os.path.join(root, "extra_submodels")
# data for 3 classes submodels
sub_train = extra_datagen.flow(subx_train, suby_train)
sub_val = extra_datagen.flow(subx_test, suby_test)

# data for 4 classes submodels 
all_train = extra_datagen.flow(x_train, y_train)
all_val = extra_datagen.flow(x_val, y_val)

# train submodels with augmented x
subclass_extra = three_class_submodel(extramodel_path, aug_train=sub_train, aug_val=sub_val)
allclass_extra = all_class_submodel(extramodel_path, aug_train=all_train, aug_val=all_val)
extra_submodels = [subclass_extra, allclass_extra]

# get stacked x for meta-learner
stacked_x_train = get_extra_stacked(x_train,extra_submodels, extra_datagen)
stacked_x_val = get_extra_stacked(x_val,extra_submodels, extra_datagen)

extra_model = get_transfer_model()
# fit model with stacked data from submodels
extra_model.fit(stacked_x_train, y_train, epochs=30, validation_data=(stacked_x_val, y_val))

Model trained with 3 classes loaded
---Sub model training---
Train with augmented data
Epoch 1/40


2022-05-10 15:09:30.053694: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


185/186 [============================>.] - ETA: 0s - loss: 1.1897 - accuracy: 0.4691

2022-05-10 15:09:32.532608: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


186/186 [==============================] - 3s 14ms/step - loss: 1.1876 - accuracy: 0.4703 - val_loss: 1.1501 - val_accuracy: 0.4866
Epoch 2/40
186/186 [==============================] - 2s 12ms/step - loss: 1.0552 - accuracy: 0.5632 - val_loss: 0.9443 - val_accuracy: 0.6301
Epoch 3/40
186/186 [==============================] - 2s 12ms/step - loss: 0.9361 - accuracy: 0.6301 - val_loss: 0.8787 - val_accuracy: 0.6397
Epoch 4/40
186/186 [==============================] - 2s 12ms/step - loss: 0.9252 - accuracy: 0.6328 - val_loss: 0.9073 - val_accuracy: 0.6382
Epoch 5/40
186/186 [==============================] - 2s 12ms/step - loss: 0.8841 - accuracy: 0.6491 - val_loss: 0.9920 - val_accuracy: 0.5867
Epoch 6/40
186/186 [==============================] - 2s 12ms/step - loss: 0.8764 - accuracy: 0.6454 - val_loss: 0.8330 - val_accuracy: 0.6534
Epoch 7/40
186/186 [==============================] - 2s 12ms/step - loss: 0.8334 - accuracy: 0.6635 - val_loss: 0.9314 - val_accuracy: 0.6013
Epoch 8/40

...

Epoch 1/30
  1/186 [..............................] - ETA: 49s - loss: 1.3935 - accuracy: 0.3750

2022-05-10 15:11:04.666412: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


183/186 [============================>.] - ETA: 0s - loss: 1.2718 - accuracy: 0.4184

2022-05-10 15:11:05.983049: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


186/186 [==============================] - 2s 8ms/step - loss: 1.2706 - accuracy: 0.4179 - val_loss: 1.1643 - val_accuracy: 0.4017
Epoch 2/30
186/186 [==============================] - 1s 7ms/step - loss: 1.0829 - accuracy: 0.4964 - val_loss: 1.0596 - val_accuracy: 0.5902
Epoch 3/30
186/186 [==============================] - 1s 7ms/step - loss: 1.0108 - accuracy: 0.6348 - val_loss: 0.9926 - val_accuracy: 0.6847
Epoch 4/30
186/186 [==============================] - 1s 7ms/step - loss: 0.9503 - accuracy: 0.7047 - val_loss: 0.9406 - val_accuracy: 0.6923
Epoch 5/30
186/186 [==============================] - 1s 7ms/step - loss: 0.9090 - accuracy: 0.7123 - val_loss: 0.9062 - val_accuracy: 0.7029
Epoch 6/30
186/186 [==============================] - 1s 6ms/step - loss: 0.8796 - accuracy: 0.7194 - val_loss: 0.8805 - val_accuracy: 0.7074
Epoch 7/30
186/186 [==============================] - 1s 7ms/step - loss: 0.8568 - accuracy: 0.7211 - val_loss: 0.8607 - val_accuracy: 0.7054
Epoch 8/30
186/18

In [62]:
pred_gen = extra_datagen.flow(x_val, shuffle = False)
pred = allclass_extra.predict_generator(pred_gen)

multiclass_classification_report(y_val, pred)

/var/folders/bc/1x56pk_j3glgqwsdp02fbltm0000gn/T/ipykernel_85409/3989295597.py:2: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  pred = allclass_extra.predict_generator(pred_gen)


              precision    recall  f1-score   support

           0       0.84      0.87      0.86       795
           1       0.58      0.73      0.65       380
           2       0.67      0.69      0.68       501
           3       0.64      0.33      0.44       303

    accuracy                           0.72      1979
   macro avg       0.68      0.66      0.66      1979
weighted avg       0.72      0.72      0.71      1979



In [59]:
# predict with mix match (avg of multiple augmentations)
meta_pred = mix_match_pred(x_test, aug_list, extra_submodels, extra_model)

# predict with augmented data
trained_pred = extra_model.predict(stacked_x_val)

multiclass_classification_report(y_test,meta_pred)
multiclass_classification_report(y_val,trained_pred)

/var/folders/bc/1x56pk_j3glgqwsdp02fbltm0000gn/T/ipykernel_85409/2772422593.py:8: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  stacked = submodel.predict_generator(pred_gen)
/var/folders/bc/1x56pk_j3glgqwsdp02fbltm0000gn/T/ipykernel_85409/2772422593.py:11: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  new_pred = submodel.predict_generator(pred_gen)
2022-05-10 15:15:45.234802: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


              precision    recall  f1-score   support

           0       0.86      0.93      0.90       803
           1       0.72      0.70      0.71       365
           2       0.67      0.87      0.76       528
           3       0.60      0.17      0.27       284

    accuracy                           0.76      1980
   macro avg       0.71      0.67      0.66      1980
weighted avg       0.75      0.76      0.73      1980

              precision    recall  f1-score   support

           0       0.82      0.93      0.87       795
           1       0.72      0.65      0.68       380
           2       0.64      0.82      0.72       501
           3       0.76      0.22      0.34       303

    accuracy                           0.74      1979
   macro avg       0.73      0.65      0.65      1979
weighted avg       0.74      0.74      0.71      1979

